<a href="https://colab.research.google.com/github/sakuragi-zero/yatteiki/blob/master/NLP/GPT2/GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#!mkdir -p '/content/drive/My Drive/NLP/GPT2rinna'
#%cd '/content/drive/My Drive/NLP/GPT2rinna'

In [ ]:
!pip install sentencepiece==0.1.91

In [ ]:
!pip install datasets==1.2.1

In [ ]:
!pip install transformers

In [ ]:
#import torch
#from transformers import T5Tokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
#tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
#model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")
#テキストの準備(2)
#text ="あそぼう"
#テキストをテンソルに変換(3)
#input = tokenizer.encode(text,return_tensors="pt")

#推論(4)
#model.eval()
#with torch.no_grad():
#  output = model.generate(input, do_sample=True, max_length=10, num_return_sequences=1)
#  print(tokenizer.batch_decode(output))

事前学習済みモデルの確認が終わったので次にデータセットの準備をします
本来なら、会話ゲームの「あかねとーく」の会話データを使う。今回は独自のデータセット  
データセットはlanguage-modelingに入れます

In [ ]:
#import random 
#ds = []
#with open("dataset_plain.txt") as f:
#  for line in f.readlines():
#    ds.append(line)

#シャッフル
#random.shuffle(ds)

#ファイルを分割して保存
#num = len(ds)
#with open("train.txt", mode="w") as f:
#  f.writelines(ds[:int(num*0.8)])
#with open("dev.txt", mode="w") as f:
# f.writelines(ds[int(num*0.8):])

学習データ(train.txt)と検証データ(dev.txt)が8:2の割合で生成されます
トークナイザーをT5に変更する

以下のコマンドでテキスト生成のファインチューニングを実行します。  
今回は事前学習済みモデルとしてrinnaの日本語GPT2モデルを使用します  
次にハイパーパラメーターの設定を行います  
train_file,validation_fileを設定します  
今回は追加でoverwrite_output_dirを追加しています。これはチューニングを実行するたびに  
outputのファイルを更新します

In [ ]:
with open("te.txt", "r") as f:
    print(f.read())

In [ ]:
%%time
# ファインチューニングの実行
!python ./transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=te.txt \
    --validation_file=te.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=20 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output/ \
    --use_fast_tokenizer=False

テンソルボードは一度使うとリロードを挟む必要があります。やり方は別途学習

In [ ]:
!nvidia-smi

--logdirの次にディレクトリーを指定します
今回はoutputディレクトリーです

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

最後にモデルを実行して成果を確かめます。  
ここでの注意点は使用するmodelがAutoModelForCausalLMであることです  
注意点を書きます。前回と違い今回実行して時に  
run-clmファイルのトークナイザーを変更せずに  
実行した場合、正しいIDが割り当てられていないため  
意味が全然通ってない言葉が出力されます。  

In [ ]:
import torch
from transformers import T5Tokenizer,AutoModelForCausalLM

text ="おはよう"
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/NLP/GPT2rinna/output")

input = tokenizer.encode(text,return_tensors="pt")

model.eval()
with torch.no_grad():
  output = model.generate(input,do_sample=True,max_length=15,num_return_sequences=1)
  print(tokenizer.batch_decode(output))

やはりデータセットを改善する方が良さそうです。  
今回は大量の文章を使いましたが次は会話セットを300万ほど準備して行いたいと思います  
その時に注意点として会話文の長さに注意した方が良さそうです。  

データの量によって一回分のバッチが足りないエラーが発生してる可能性があるのでデータセットの量を拡張する